# Stock Market Analysis and Prediction Project
## Exploratory Data Analysis

This notebook performs exploratory data analysis on the stock market data collected from Yahoo Finance API. We'll analyze price trends, calculate financial metrics, and create visualizations to gain insights into the data.

In [ ]:
# Import necessary libraries
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Set plot styles
plt.style.use('fivethirtyeight')
sns.set_theme(style="darkgrid")
%matplotlib inline

# Increase plot size
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Define the database path
DB_PATH = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'data', 'stock_data.db')
print(f"Database path: {DB_PATH}")

# Function to connect to the database
def get_connection():
    return sqlite3.connect(DB_PATH)

# Function to get stock data from the database
def get_stock_data(symbol=None):
    conn = get_connection()
    
    if symbol:
        query = """
        SELECT s.symbol, s.company_name, p.date, p.open, p.high, p.low, p.close, p.adj_close, p.volume
        FROM stocks s
        JOIN stock_prices p ON s.id = p.stock_id
        WHERE s.symbol = ?
        ORDER BY p.date
        """
        df = pd.read_sql_query(query, conn, params=(symbol,))
    else:
        query = """
        SELECT s.symbol, s.company_name, p.date, p.open, p.high, p.low, p.close, p.adj_close, p.volume
        FROM stocks s
        JOIN stock_prices p ON s.id = p.stock_id
        ORDER BY s.symbol, p.date
        """
        df = pd.read_sql_query(query, conn)
    
    conn.close()
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    return df

# Function to get technical indicators from the database
def get_technical_indicators(symbol):
    conn = get_connection()
    
    query = """
    SELECT s.symbol, t.date, t.sma_20, t.sma_50, t.sma_200, t.rsi_14, t.macd, t.macd_signal,
           t.bollinger_upper, t.bollinger_middle, t.bollinger_lower
    FROM stocks s
    JOIN technical_indicators t ON s.id = t.stock_id
    WHERE s.symbol = ?
    ORDER BY t.date
    """
    
    df = pd.read_sql_query(query, conn, params=(symbol,))
    conn.close()
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    return df

# Get list of available stocks
def get_stock_list():
    conn = get_connection()
    query = "SELECT symbol, company_name FROM stocks ORDER BY symbol"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [ ]:
# Get list of available stocks
stocks_df = get_stock_list()
stocks_df

## 1. Overview of Stock Price Data

Let's first look at the closing prices of all stocks over time to get a general overview of their performance.

In [ ]:
# Get all stock data
all_stocks_df = get_stock_data()

# Display the first few rows
all_stocks_df.head()

In [ ]:
# Check for missing values
print("Missing values in the dataset:")
all_stocks_df.isnull().sum()

In [ ]:
# Plot closing prices for all stocks
plt.figure(figsize=(16, 10))

# Create a pivot table with dates as index and symbols as columns
pivot_df = all_stocks_df.pivot_table(index='date', columns='symbol', values='close')

# Plot each stock
for column in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[column], label=column)

plt.title('Stock Closing Prices Over Time', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Closing Price (USD)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/all_stocks_closing_prices.png')
plt.show()

The above plot shows the raw closing prices, but it's difficult to compare stocks with very different price ranges. Let's normalize the prices to see relative performance.

In [ ]:
# Normalize prices (percentage change from first day)
normalized_df = pivot_df.copy()
for column in normalized_df.columns:
    normalized_df[column] = normalized_df[column] / normalized_df[column].iloc[0] * 100

# Plot normalized prices
plt.figure(figsize=(16, 10))
for column in normalized_df.columns:
    plt.plot(normalized_df.index, normalized_df[column], label=column)

plt.title('Normalized Stock Performance (Base 100)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Normalized Price (%)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/normalized_stock_performance.png')
plt.show()

## 2. Detailed Analysis of Individual Stocks

Let's analyze each stock in more detail, starting with Netflix (NFLX) as an example.

In [ ]:
# Get Netflix data
nflx_df = get_stock_data('NFLX')
nflx_df.head()

In [ ]:
# Plot NFLX price with volume
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.1, subplot_titles=('NFLX Price', 'Volume'),
                    row_heights=[0.7, 0.3])

# Add price candlestick chart
fig.add_trace(go.Candlestick(x=nflx_df['date'],
                            open=nflx_df['open'],
                            high=nflx_df['high'],
                            low=nflx_df['low'],
                            close=nflx_df['close'],
                            name='Price'),
              row=1, col=1)

# Add volume bar chart
fig.add_trace(go.Bar(x=nflx_df['date'], y=nflx_df['volume'], name='Volume', marker_color='rgba(0, 0, 255, 0.5)'),
              row=2, col=1)

# Update layout
fig.update_layout(title='Netflix (NFLX) Stock Price and Volume',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)',
                  xaxis_rangeslider_visible=False,
                  height=800,
                  width=1200)

fig.write_html('../visualizations/nflx_price_volume.html')
fig.show()

In [ ]:
# Get technical indicators for NFLX
nflx_indicators = get_technical_indicators('NFLX')
nflx_indicators.head()

In [ ]:
# Merge price data with technical indicators
nflx_merged = pd.merge(nflx_df, nflx_indicators, on=['symbol', 'date'])
nflx_merged.head()

In [ ]:
# Plot NFLX price with moving averages
plt.figure(figsize=(16, 10))
plt.plot(nflx_merged['date'], nflx_merged['close'], label='Close Price', alpha=0.7)
plt.plot(nflx_merged['date'], nflx_merged['sma_20'], label='20-day SMA', alpha=0.7)
plt.plot(nflx_merged['date'], nflx_merged['sma_50'], label='50-day SMA', alpha=0.7)
plt.plot(nflx_merged['date'], nflx_merged['sma_200'], label='200-day SMA', alpha=0.7)

plt.title('Netflix (NFLX) Stock Price with Moving Averages', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Price (USD)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/nflx_moving_averages.png')
plt.show()

In [ ]:
# Plot NFLX with Bollinger Bands
plt.figure(figsize=(16, 10))
plt.plot(nflx_merged['date'], nflx_merged['close'], label='Close Price', color='blue', alpha=0.7)
plt.plot(nflx_merged['date'], nflx_merged['bollinger_upper'], label='Upper Band', color='red', alpha=0.5)
plt.plot(nflx_merged['date'], nflx_merged['bollinger_middle'], label='Middle Band', color='green', alpha=0.5)
plt.plot(nflx_merged['date'], nflx_merged['bollinger_lower'], label='Lower Band', color='red', alpha=0.5)

# Fill between upper and lower bands
plt.fill_between(nflx_merged['date'], nflx_merged['bollinger_upper'], nflx_merged['bollinger_lower'], 
                 color='gray', alpha=0.2)

plt.title('Netflix (NFLX) Stock Price with Bollinger Bands', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Price (USD)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/nflx_bollinger_bands.png')
plt.show()

In [ ]:
# Plot RSI for NFLX
plt.figure(figsize=(16, 6))
plt.plot(nflx_merged['date'], nflx_merged['rsi_14'], label='RSI-14', color='purple')
plt.axhline(y=70, color='red', linestyle='--', alpha=0.5)
plt.axhline(y=30, color='green', linestyle='--', alpha=0.5)
plt.fill_between(nflx_merged['date'], nflx_merged['rsi_14'], 70, where=(nflx_merged['rsi_14'] >= 70), color='red', alpha=0.3)
plt.fill_between(nflx_merged['date'], nflx_merged['rsi_14'], 30, where=(nflx_merged['rsi_14'] <= 30), color='green', alpha=0.3)

plt.title('Netflix (NFLX) Relative Strength Index (RSI-14)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('RSI Value', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/nflx_rsi.png')
plt.show()

In [ ]:
# Plot MACD for NFLX
plt.figure(figsize=(16, 6))
plt.plot(nflx_merged['date'], nflx_merged['macd'], label='MACD', color='blue')
plt.plot(nflx_merged['date'], nflx_merged['macd_signal'], label='Signal Line', color='red')
plt.bar(nflx_merged['date'], nflx_merged['macd'] - nflx_merged['macd_signal'], 
        label='Histogram', color=['green' if val >= 0 else 'red' for val in (nflx_merged['macd'] - nflx_merged['macd_signal'])], alpha=0.5)

plt.title('Netflix (NFLX) MACD Indicator', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('MACD Value', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/nflx_macd.png')
plt.show()

## 3. Comparative Analysis of Tech Stocks

Let's compare the performance of tech stocks in our dataset (AAPL, AMZN, GOOGL, META, MSFT, NFLX, TSLA).

In [ ]:
# Filter for tech stocks
tech_stocks = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NFLX', 'TSLA']
tech_df = all_stocks_df[all_stocks_df['symbol'].isin(tech_stocks)]

# Create a pivot table with dates as index and symbols as columns
tech_pivot = tech_df.pivot_table(index='date', columns='symbol', values='close')

# Normalize prices (percentage change from first day)
tech_normalized = tech_pivot.copy()
for column in tech_normalized.columns:
    tech_normalized[column] = tech_normalized[column] / tech_normalized[column].iloc[0] * 100

# Plot normalized prices
plt.figure(figsize=(16, 10))
for column in tech_normalized.columns:
    plt.plot(tech_normalized.index, tech_normalized[column], label=column)

plt.title('Normalized Tech Stock Performance (Base 100)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Normalized Price (%)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/tech_stocks_normalized.png')
plt.show()

In [ ]:
# Calculate daily returns
tech_returns = tech_pivot.pct_change().dropna()

# Plot correlation heatmap of daily returns
plt.figure(figsize=(12, 10))
correlation_matrix = tech_returns.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Correlation of Daily Returns Among Tech Stocks', fontsize=16)
plt.tight_layout()
plt.savefig('../visualizations/tech_stocks_correlation.png')
plt.show()

In [ ]:
# Calculate volatility (standard deviation of returns)
volatility = tech_returns.std() * np.sqrt(252)  # Annualized volatility

# Calculate average annual return
annual_return = tech_returns.mean() * 252  # Annualized return

# Create a DataFrame for risk-return analysis
risk_return = pd.DataFrame({
    'Volatility (Risk)': volatility,
    'Annual Return': annual_return
})

# Plot risk-return scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(risk_return['Volatility (Risk)'], risk_return['Annual Return'], s=100)

# Add labels for each stock
for i, stock in enumerate(risk_return.index):
    plt.annotate(stock, 
                 (risk_return['Volatility (Risk)'][i], risk_return['Annual Return'][i]),
                 xytext=(5, 5), textcoords='offset points', fontsize=12)

plt.title('Risk-Return Profile of Tech Stocks', fontsize=16)
plt.xlabel('Volatility (Risk)', fontsize=14)
plt.ylabel('Annual Return', fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/tech_stocks_risk_return.png')
plt.show()

# Display the risk-return data
risk_return.sort_values('Annual Return', ascending=False)

## 4. Sector Analysis

Let's group stocks by sector and analyze sector performance.

In [ ]:
# Get sector information
conn = get_connection()
sector_query = "SELECT symbol, sector FROM stocks"
sector_df = pd.read_sql_query(sector_query, conn)
conn.close()

# Display sectors
sector_df

In [ ]:
# Group stocks by sector
sectors = {}
for sector in sector_df['sector'].unique():
    sectors[sector] = sector_df[sector_df['sector'] == sector]['symbol'].tolist()

# Create a DataFrame with sector average performance
sector_performance = pd.DataFrame(index=pivot_df.index)

for sector, symbols in sectors.items():
    # Filter pivot_df for symbols in this sector
    sector_stocks = pivot_df[symbols]
    
    # Normalize each stock
    normalized_sector = sector_stocks.copy()
    for column in normalized_sector.columns:
        normalized_sector[column] = normalized_sector[column] / normalized_sector[column].iloc[0] * 100
    
    # Calculate sector average
    sector_performance[sector] = normalized_sector.mean(axis=1)

# Plot sector performance
plt.figure(figsize=(16, 10))
for column in sector_performance.columns:
    plt.plot(sector_performance.index, sector_performance[column], label=column, linewidth=2)

plt.title('Normalized Sector Performance (Base 100)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Normalized Price (%)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/sector_performance.png')
plt.show()

## 5. Volume Analysis

Let's analyze trading volume patterns across stocks.

In [ ]:
# Create a pivot table for volume
volume_pivot = all_stocks_df.pivot_table(index='date', columns='symbol', values='volume')

# Calculate average daily volume for each stock
avg_volume = volume_pivot.mean()
avg_volume_df = pd.DataFrame({'Average Daily Volume': avg_volume}).sort_values('Average Daily Volume', ascending=False)

# Plot average daily volume
plt.figure(figsize=(14, 8))
plt.bar(avg_volume_df.index, avg_volume_df['Average Daily Volume'], color='skyblue')
plt.title('Average Daily Trading Volume by Stock', fontsize=16)
plt.xlabel('Stock Symbol', fontsize=14)
plt.ylabel('Average Volume (Shares)', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig('../visualizations/average_daily_volume.png')
plt.show()

# Display the average volume data
avg_volume_df

In [ ]:
# Plot volume trends over time for top 5 stocks by volume
top_volume_stocks = avg_volume_df.head(5).index.tolist()
volume_subset = volume_pivot[top_volume_stocks]

# Resample to monthly for clearer visualization
monthly_volume = volume_subset.resample('M', on=volume_subset.index).mean()

plt.figure(figsize=(16, 10))
for column in monthly_volume.columns:
    plt.plot(monthly_volume.index, monthly_volume[column], label=column, linewidth=2)

plt.title('Monthly Average Trading Volume for Top 5 Stocks', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Volume (Shares)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/monthly_volume_trends.png')
plt.show()

## 6. Volatility Analysis

Let's analyze the volatility of each stock over time.

In [ ]:
# Calculate daily returns for all stocks
returns = pivot_df.pct_change().dropna()

# Calculate 30-day rolling volatility (annualized)
volatility_30d = returns.rolling(window=30).std() * np.sqrt(252)

# Plot 30-day rolling volatility for all stocks
plt.figure(figsize=(16, 10))
for column in volatility_30d.columns:
    plt.plot(volatility_30d.index, volatility_30d[column], label=column)

plt.title('30-Day Rolling Volatility (Annualized)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Volatility', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/rolling_volatility.png')
plt.show()

In [ ]:
# Calculate average volatility for each stock
avg_volatility = volatility_30d.mean().sort_values(ascending=False)
avg_volatility_df = pd.DataFrame({'Average Volatility': avg_volatility})

# Plot average volatility
plt.figure(figsize=(14, 8))
plt.bar(avg_volatility_df.index, avg_volatility_df['Average Volatility'], color='salmon')
plt.title('Average Volatility by Stock', fontsize=16)
plt.xlabel('Stock Symbol', fontsize=14)
plt.ylabel('Average Volatility', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig('../visualizations/average_volatility.png')
plt.show()

# Display the average volatility data
avg_volatility_df

## 7. Seasonal Analysis

Let's analyze seasonal patterns in stock returns.

In [ ]:
# Add month and year columns to returns DataFrame
returns_with_date = returns.copy()
returns_with_date['month'] = returns_with_date.index.month
returns_with_date['year'] = returns_with_date.index.year

# Calculate average monthly returns for each stock
monthly_returns = {}
for symbol in returns.columns:
    monthly_avg = returns_with_date.groupby('month')[symbol].mean() * 100  # Convert to percentage
    monthly_returns[symbol] = monthly_avg

# Convert to DataFrame
monthly_returns_df = pd.DataFrame(monthly_returns)
monthly_returns_df.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Plot heatmap of monthly returns
plt.figure(figsize=(16, 10))
sns.heatmap(monthly_returns_df, annot=True, cmap='RdYlGn', center=0, fmt='.2f')
plt.title('Average Monthly Returns by Stock (%)', fontsize=16)
plt.tight_layout()
plt.savefig('../visualizations/monthly_returns_heatmap.png')
plt.show()

In [ ]:
# Calculate average market return (equal-weighted portfolio)
returns_with_date['Market'] = returns.mean(axis=1)
market_monthly_avg = returns_with_date.groupby('month')['Market'].mean() * 100  # Convert to percentage
market_monthly_avg.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Plot average market monthly returns
plt.figure(figsize=(14, 8))
colors = ['green' if x >= 0 else 'red' for x in market_monthly_avg]
plt.bar(market_monthly_avg.index, market_monthly_avg, color=colors)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.title('Average Market Monthly Returns (%)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Average Return (%)', fontsize=14)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig('../visualizations/market_monthly_returns.png')
plt.show()

## 8. Summary of Findings

Based on our exploratory data analysis, here are the key findings:

1. **Overall Performance**: 
   - Tech stocks generally outperformed other sectors during the analyzed period.
   - [Stock with highest return] showed the best performance with a [X]% increase.
   - [Stock with lowest return] had the weakest performance with a [Y]% change.

2. **Volatility**:
   - [Most volatile stock] exhibited the highest volatility.
   - [Least volatile stock] was the most stable stock in our dataset.
   - Market volatility peaked during [specific periods], likely due to [potential reasons].

3. **Correlations**:
   - Stocks within the same sector showed stronger correlations.
   - [Specific stocks] showed unusually high correlation despite being in different sectors.
   - [Specific stocks] showed low correlation with the rest of the market, suggesting potential diversification benefits.

4. **Seasonal Patterns**:
   - [Specific months] typically showed stronger returns across most stocks.
   - [Specific months] were generally weaker for stock performance.
   - [Specific stock] showed unique seasonal patterns compared to the broader market.

5. **Trading Volume**:
   - [Specific stock] consistently had the highest trading volume.
   - Volume spikes were observed during [specific events or periods].
   - There appears to be a [positive/negative/no] relationship between volume and price movements.

6. **Technical Indicators**:
   - Moving averages identified key support and resistance levels for [specific stocks].
   - RSI indicated overbought conditions for [specific stocks] during [specific periods].
   - MACD crossovers provided potential trading signals that aligned with major price movements.

These findings provide valuable insights for developing predictive models in the next phase of our analysis.

## Next Steps

Based on our exploratory data analysis, we'll proceed with the following steps:

1. **Feature Engineering**: Create additional features based on the insights gained from this analysis.
2. **Model Development**: Build predictive models for stock price forecasting.
3. **Risk Analysis**: Develop a risk assessment framework based on volatility and correlation analysis.
4. **Portfolio Optimization**: Create an optimal portfolio allocation strategy based on risk-return profiles.
5. **Interactive Dashboard**: Develop a Power BI dashboard to visualize the findings and predictions.